###Chcek notebook's internet speed

In [ ]:
!sudo apt install speedtest-cli
!speedtest-cli

###Time out preventer

Run this code to prevent Notebook Timeout

In [ ]:
from os import makedirs
makedirs("/root/.config/rclone", exist_ok = True)
import IPython
from google.colab import output

display(IPython.display.Javascript('''
function ClickConnect(){
  btn = document.querySelector("colab-connect-button")
  if (btn != null){
    console.log("Click colab-connect-button"); 
    btn.click() 
    }
    
    btn = document.getElementById('ok')
    if (btn != null){
    console.log("Click reconnect"); 
    btn.click() 
    }
  }
  
setInterval(ClickConnect,60000)
'''))

### Start session

In [ ]:

!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent

!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)

sett = {'allow_multiple_connections_per_ip': True,
  'dont_count_slow_torrents': True,
  'active_downloads': -1,
  'active_seeds': 30,
  'active_limit':100}

ses.apply_settings(sett)

downloads = []

###Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

###Add torrent file

In [ ]:
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Library/Downloads",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

###Add torrent magnet

In [ ]:
params = {"save_path": "/content/drive/My Drive/Library/Downloads"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

###Start Torrent

In [ ]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)